### Libraries and Device

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import os
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
from torchvision import transforms
from tqdm import tqdm
import traceback

from models.MultiStage_Composite_V4 import CompositeModel
from util import preprocess_image

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"[INFO] Using device: {device}")

[INFO] Using device: cuda


### Model and Settings

In [ ]:
# Paths
images_folder = os.path.join("..", "..", "CV_data", "miniImageNet")  # Path to images
qwen_embeddings_path = os.path.join("..", "..", "CV_data", "separated_embeddings", "qwen_384_test")  # Path to Qwen embeddings
test_images_csv = os.path.join("..", "assets", "test_images.csv")  # CSV file with test image filenames and labels
composite_model_state_path = os.path.join("..", "models", "V5_finetuning_adapter", "best_model_epoch_2_loss_2.0951.pth")  # Path to the trained composite model

# Model
model = CompositeModel().to(device).half()
composite_model_state_dict = torch.load(composite_model_state_path, weights_only=False)['model_state_dict']
model.load_state_dict(composite_model_state_dict)
print("[INFO] Composite model loaded.")

# Image settings
image_size = 384

[INFO] Composite model loaded.


### Test

In [ ]:
# Loss
loss_function = nn.MSELoss()

# Image names list from CSV
df = pd.read_csv(test_images_csv)
image_names = df['filename'].tolist()

# Loss history
loss_history = []

# Loop through images and extract embeddings
for img_name in tqdm(image_names, desc="[LOOP] Processing images"):
    try:
        # Load and preprocess image
        img_path = os.path.join(images_folder, img_name)
        image = preprocess_image(img_path, image_size)
        transform = transforms.Compose([transforms.ToTensor(),])
        resnet_inputs = transform(image).unsqueeze(0).to(device, torch.float16)

        # Extract Qwen embeddings
        with torch.no_grad():
            composite_model_output = model(resnet_inputs)

        # load qwen emb from qwen_embeddings_path
        qwen_emb = torch.load(os.path.join(qwen_embeddings_path, f"{os.path.splitext(img_name)[0]}.pt")).to(device, torch.float16)
        
        # Compute loss
        loss = loss_function(composite_model_output, qwen_emb)
        loss_history.append(loss.item())

        # Free up memory
        del composite_model_output, qwen_emb, resnet_inputs
        torch.cuda.empty_cache()

    except Exception as e:
        print(f"[ERROR] Error processing {img_name}: {e}")
        traceback.print_exc()

# Print average loss and standard deviation
if loss_history:
    avg_loss = np.nanmean(loss_history)
    print(f"\n[INFO] Average Loss: {avg_loss:.4f}")
    std_loss = np.nanstd(loss_history)
    print(f"[INFO] Standard Deviation: {std_loss:.4f}")
else:
    print("[INFO] No losses recorded.")

[LOOP] Processing images: 100%|██████████| 9001/9001 [02:54<00:00, 51.71it/s]


[INFO] Average Loss: 2.3568
[INFO] Standard Deviation: 0.5679
